In [56]:
!pip install torch_geometric
!pip install spektral

In [57]:
import numpy as np
import tensorflow as tf
import torch as th
from spektral.layers.convolutional import ARMAConv as ARMAConvSpektral
from torch_geometric.nn import ARMAConv as ARMAConvTorch
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader
from torch.utils.data import TensorDataset



In [58]:

# Create a sample adjacency matrix (replace this with your actual data)
adjacency_matrix = np.array([[[0, 1, 0],
                             [1, 0, 1],
                             [0, 1, 0]],[[0, 1, 0],
                             [1, 0, 1],
                             [0, 1, 0]]])

# Save the adjacency matrix as adjacency_df.npy
np.save("adjacency_df.npy", adjacency_matrix)
a_input = np.load('adjacency_df.npy')

In [59]:
a_input

array([[[0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]],

       [[0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]]])

In [60]:
def array_matrices_to_array_edge_index(array_matrices: np.ndarray):
    print(array_matrices.shape)
    array_edge_indices = []
    array_edge_indices_weighted = []

    # Iterate over the adjacency matrices
    for matrix in array_matrices:
        edge_indices = []
        edge_weights = []
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if matrix[i, j] != 0:
                    # Add the edge indices and the weight
                    edge_indices.append([i, j])
                    edge_weights.append(matrix[i, j])

        #the edge indices and weights should always contain 3 columns, fill with [] if not

        edge_indices = np.array(edge_indices)
        edge_weights = np.array(edge_weights)
        array_edge_indices.append(edge_indices)
        array_edge_indices_weighted.append(edge_weights)

    return np.array(array_edge_indices), np.array(array_edge_indices_weighted)

In [61]:
array_matrices_to_array_edge_index(a_input[1:2])

(1, 3, 3)


(array([[[0, 1],
         [1, 0],
         [1, 2],
         [2, 1]]]),
 array([[1, 1, 1, 1]]))

In [62]:
node_features = th.tensor([
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
])

edge_indices = th.tensor(
    [
        [0, 1, 1, 1],  # Source nodes
        [0, 2, 0, 1],  # Target nodes
    ]

)



# Adjacency matrix
# [1., 3., 0.],
# [3., 4., 2.],
# [0., 2., 0.],


edge_weights = th.tensor([
    1., 2., 3., 4.
])

print(node_features.shape)
print(edge_indices.shape)
print(edge_weights.shape)

arma = ARMAConvTorch(3, 3, shared_weights=True)
arma(node_features, edge_indices, edge_weight=edge_weights)

torch.Size([3, 3])
torch.Size([2, 4])
torch.Size([4])


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<MeanBackward1>)

In [63]:
# node_features = tf.convert_to_tensor([
#     [
#         [1., 0., 0.],
#         [0., 0., 0.],
#         [0., 0., 0.],
#     ],
#     [
#         [0., 0., 0.],
#         [0., 2., 0.],
#         [0., 0., 0.],
#     ]
# ]
#
# )
#
adjacency_graph = tf.convert_to_tensor(
    np.array([
        [
            [1., 0., 0.,2.],
            [0., 1., 0.,2.],
            [0., 0., 1.,2.]
        ],
        [
            [0., 2., 3.,5.],
            [2., 0., 4.,0.],
            [3., 4., 0.,0.],

        ],
    ]
    )
)

node_features = tf.convert_to_tensor(
    [
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
    ]
)

adjacency_graph = tf.convert_to_tensor(

    [
        [1., 3., 0.],
        [3., 4., 2.],
        [0., 2., 0.],
    ],

)

print(node_features.shape)
print(adjacency_graph.shape)

arma = ARMAConvSpektral(node_features.shape[-1], shared_weights=True)
arma([node_features, adjacency_graph])

(3, 3)
(3, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [64]:
import torch
from torch_geometric.nn import ARMAConv
from torch_geometric.data import Data

# Sample data preparation
num_nodes = 10
num_node_features = 5
num_classes = 3
edges = torch.tensor([[0, 1, 1, 2,2],
                      [1, 2, 0, 3,1]], dtype=torch.long)
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])

# Graph data object
data = Data(x=node_features, edge_index=edges)


# Model
class PyGARMA(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(num_node_features, num_classes, num_stacks=1, num_layers=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        return x


model_pyg = PyGARMA()
model_pyg.train()
out = model_pyg(data)
print(out)

tensor([[2.5138, 0.6672, 1.5112],
        [6.6435, 1.4134, 0.0000],
        [3.8216, 0.0000, 0.6397],
        [3.1332, 0.5340, 1.3213],
        [0.0000, 0.0000, 3.8102],
        [0.0000, 0.0000, 3.8102],
        [0.0000, 0.0000, 3.8102],
        [0.0000, 0.0000, 3.8102],
        [0.0000, 0.0000, 3.8102],
        [0.0000, 0.0000, 3.8102]], grad_fn=<MeanBackward1>)


In [65]:
import numpy as np
from spektral.layers import ARMAConv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from spektral.data import Graph

# Converting PyTorch data to Spektral graph
num_nodes = 10
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])
x = np.array(node_features)
a = np.zeros((num_nodes, num_nodes))
a[edges[0], edges[1]] = 1

graph = Graph(x=x, a=a)


# Spektral model
class SpektralARMA(Model):

    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(channels=num_classes, n_stacks=1, n_layers=1)

    def call(self, inputs):
        x, a = inputs
        return self.arma([x, a])


x_in = Input(shape=(num_nodes,num_node_features))
a_in = Input((num_nodes,num_nodes))
model_spektral = SpektralARMA()

print(x_in.shape)
print(a_in.shape)
model_spektral([x_in, a_in])
model_spektral.compile()
out = model_spektral([graph.x, graph.a])
print(out)


(None, 10, 5)
(None, 10, 10)
tf.Tensor(
[[3.1539402  1.9334886  0.        ]
 [3.304453   3.2534823  0.        ]
 [1.8779955  3.6067314  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]
 [0.15051264 1.3199936  0.        ]], shape=(10, 3), dtype=float32)


In [66]:
import torch

# Example adjacency matrices as dense tensors
adj_matrices = torch.tensor([
    [[1., 0., 0.],
     [0., 1., 1.],
     [0., 1., 1.]],

    [[0., 1., 0.],
     [1., 0., 1.],
     [0., 1., 0.]]
])

# Convert adjacency matrices to edge_index format and edge_type
num_nodes = adj_matrices.size(1)
edge_index = []
edge_type = []

for i, adj in enumerate(adj_matrices):
    row, col = torch.where(adj > 0)
    edge_index.append(torch.stack([row, col], dim=0))
    edge_type.append(torch.full((row.size(0),), i, dtype=torch.long))

# Concatenate all edges
edge_index = torch.cat(edge_index, dim=1)
edge_type = torch.cat(edge_type)

print(edge_index)
print(edge_type)


tensor([[0, 1, 1, 2, 2, 0, 1, 1, 2],
        [0, 1, 2, 1, 2, 1, 0, 2, 1]])
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1])


In [106]:
x = torch.randn(5,5, 3)
y = torch.randn(5,5, 3)
input_train_tensor = [torch.tensor(data, dtype=torch.float32) for data in x]
input_test_tensor = [torch.tensor(data, dtype=torch.float32) for data in y]
data_list = []
for i in range(len(input_train_tensor)):
    num_nodes = input_train_tensor[i].shape[0]
    edge_index = torch.randint(0, num_nodes, (2, num_nodes), dtype=torch.long)
    data = Data(x=input_train_tensor[i], edge_index=edge_index, y=input_test_tensor[i])

    data_list.append(data)




<ipython-input-106-b44bbd7ed8ac>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_train_tensor = [torch.tensor(data, dtype=torch.float32) for data in x]
<ipython-input-106-b44bbd7ed8ac>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_test_tensor = [torch.tensor(data, dtype=torch.float32) for data in y]


In [107]:

batch_size = 2
train_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
a = 1
for batch in train_loader:
    print(f"Batch: {a}")
    print("x:", batch.x)
    print("y:", batch.y)
    print("edge_index:", batch.edge_index)
    print("Number Nodes:", batch.num_nodes)
    print("Number of graphs in the batch:", batch.num_graphs)
    print("\n\n")

    a = a + 1

Batch: 1
x: tensor([[-0.2613,  0.8541, -1.0763],
        [-1.0057,  1.1086,  0.1031],
        [ 1.0287, -0.8067,  0.2869],
        [ 0.7622,  1.1406,  0.3393],
        [-0.1539,  0.8585,  0.5838],
        [ 0.4280,  0.6156, -0.1647],
        [ 0.7239, -0.9180, -0.5872],
        [-0.3697, -0.4415, -0.1578],
        [ 1.2144, -0.4634,  2.1939],
        [ 0.0091,  0.6681, -0.1492]])
y: tensor([[-0.5355, -0.6231, -1.5089],
        [-0.5277, -0.6746, -1.2371],
        [-1.3628,  1.2023, -0.7898],
        [ 0.4992,  0.1609, -0.1483],
        [-0.2168,  0.7827, -2.2142],
        [ 0.6048,  0.7583,  0.9721],
        [ 1.2337,  0.3700,  0.0603],
        [-0.4360,  0.5730, -0.1816],
        [-1.2227, -2.1468, -0.5743],
        [-0.7404,  1.3041, -1.1219]])
edge_index: tensor([[3, 1, 3, 3, 2, 7, 7, 9, 5, 8],
        [1, 4, 2, 1, 0, 7, 8, 8, 9, 6]])
Number Nodes: 10
Number of graphs in the batch: 2



Batch: 2
x: tensor([[ 1.0538,  1.4814, -0.3034],
        [ 0.9929,  0.4036,  0.1623],
        [-0

In [97]:
batch_size = len(input_train_tensor)
train_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
a = 1
for batch in train_loader:
    print(f"Batch: {a}")
    print("x:", batch.x)
    print("y:", batch.y)
    print("edge_index:", batch.edge_index)
    print("Number Nodes:", batch.num_nodes)
    print("Number of graphs in the batch:", batch.num_graphs)
    print("\n\n")

    a = a + 1

Batch: 1
x: tensor([0., 3., 0., 2., 0., 0., 0., 0., 1.])
y: tensor([0.3809, 0.2574, 0.4651, 0.5635, 0.3809, 0.0000, 0.0000, 0.4651, 0.4727])
edge_index: tensor([[0, 0, 1, 4, 4, 5, 8, 6, 8],
        [2, 1, 2, 3, 3, 5, 6, 6, 8]])
Number Nodes: 9
Number of graphs in the batch: 3



